In [ ]:
# ==============================
# 1. IMPORTS
# ==============================
import os
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# ==============================
# 2. PARAMETERS
# ==============================
train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
external_test_dir = '/kaggle/input/asl-alphabet-test'

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

num_classes = len(classes)
learning_rate = 1e-4 
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20

# ==============================
# 3. DATA GENERATORS (RESNET PREPROCESSING)
# ==============================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # ResNet specific preprocessing (-1 to 1)
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,   # Keep False for ASL
    fill_mode='nearest',
    validation_split=0.1
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# ==============================
# 4. MODEL CREATION (RESNET50V2)
# ==============================
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze bottom 90%, Unfreeze top 10%
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.9)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

adam = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ==============================
# 5. TRAINING WITH CALLBACKS
# ==============================
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    min_lr=1e-6, 
    verbose=1
)

start = time()
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)
train_time = time() - start
print(f"\nTraining completed in {train_time:.2f} seconds.")

model.save("asl_resnet50v2_optimized.keras")
print("Model saved successfully as 'asl_resnet50v2_optimized.keras'!")

# ==============================
# 6. PLOT TRAINING HISTORY (NEW SECTION)
# ==============================
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.show()

plot_history(history)

# ==============================
# 7. EVALUATION ON VALIDATION SET
# ==============================
def evaluate_generator(generator, title="Dataset"):
    loss, acc = model.evaluate(generator, verbose=1)
    print(f"\n{title} - Accuracy: {acc:.4f}, Loss: {loss:.4f}")

    y_pred = model.predict(generator, verbose=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = generator.classes

    cm = confusion_matrix(y_true_labels, y_pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    plt.figure(figsize=(14, 14))
    disp.plot(xticks_rotation='vertical', cmap='Blues', ax=plt.gca())
    plt.title(f"Confusion Matrix - {title}")
    plt.show()

    print(f"\nCLASSIFICATION REPORT - {title}")
    print(classification_report(y_true_labels, y_pred_labels, target_names=classes))

evaluate_generator(val_generator, title="Validation Set")

# ==============================
# 8. LOAD EXTERNAL TEST DATA
# ==============================
def load_external_test_data(test_dir, img_size=(128, 128)):
    images, labels = [], []
    
    if not os.path.exists(test_dir):
        print(f"Error: Directory {test_dir} not found.")
        return np.array([]), np.array([])

    for folder in os.listdir(test_dir):
        if folder not in classes:
            continue
            
        label_idx = classes.index(folder)
        folder_path = os.path.join(test_dir, folder)
        
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            
            img = cv2.imread(img_path)
            
            if img is None:
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)
            
            # ResNet Preprocessing
            img = img.astype('float32')
            img = preprocess_input(img) 
            
            images.append(img)
            labels.append(label_idx)
            
    images = np.array(images)
    labels = to_categorical(labels, num_classes=num_classes)
    
    print(f"\nLoaded {images.shape[0]} external test images. Shape = {images.shape}")
    return images, labels

x_test_external, y_test_external = load_external_test_data(external_test_dir, img_size=input_shape[:2])

# ==============================
# 9. EVALUATE ON EXTERNAL TEST SET
# ==============================
def evaluate_on_arrays(x_data, y_data, title="External Test Set"):
    if x_data.size == 0:
        print("No data to evaluate.")
        return

    loss, acc = model.evaluate(x_data, y_data, verbose=1)
    print(f"\n{title} - Accuracy: {acc:.4f}, Loss: {loss:.4f}")

    y_pred = model.predict(x_data, verbose=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_data, axis=1)

    cm = confusion_matrix(y_true_labels, y_pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    plt.figure(figsize=(14, 14))
    disp.plot(xticks_rotation='vertical', cmap='Blues', ax=plt.gca())
    plt.title(f"Confusion Matrix - {title}")
    plt.show()

    print(f"\nCLASSIFICATION REPORT - {title}")
    print(classification_report(y_true_labels, y_pred_labels, target_names=classes))

evaluate_on_arrays(x_test_external, y_test_external, title="External Test Set")

In [ ]:
# ==============================
# 1. IMPORTS
# ==============================
import os
import cv2
import numpy as np
from time import time
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split


In [ ]:
# ==============================
# 2. PARAMETERS
# ==============================
train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
external_test_dir = '/kaggle/input/asl-alphabet-test'

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

num_classes = len(classes)
learning_rate = 1e-4 
input_shape = (128, 128, 3)
batch_size = 32
epochs = 20


In [ ]:
# ==============================
# 3. DATA GENERATORS (RESNET PREPROCESSING)
# ==============================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # ResNet specific preprocessing (-1 to 1)
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,   # Keep False for ASL
    fill_mode='nearest',
    validation_split=0.1
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


In [ ]:
# ==============================
# 4. MODEL CREATION (RESNET50V2)
# ==============================
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze bottom 90%, Unfreeze top 10%
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.9)

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

adam = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# ==============================
# 5. TRAINING WITH CALLBACKS
# ==============================
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    min_lr=1e-6, 
    verbose=1
)

start = time()
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)
train_time = time() - start
print(f"\nTraining completed in {train_time:.2f} seconds.")

model.save("asl_resnet50v2_optimized.keras")
print("Model saved successfully as 'asl_resnet50v2_optimized.keras'!")


In [ ]:
model.save("asl_resnet50v2_optimized.h5")

In [ ]:
# ==============================
# 6. PLOT TRAINING HISTORY (NEW SECTION)
# ==============================
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.show()

plot_history(history)


In [ ]:
# ==============================
# 7. EVALUATION ON VALIDATION SET
# ==============================
def evaluate_generator(generator, title="Dataset"):
    loss, acc = model.evaluate(generator, verbose=1)
    print(f"\n{title} - Accuracy: {acc:.4f}, Loss: {loss:.4f}")

    y_pred = model.predict(generator, verbose=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = generator.classes

    cm = confusion_matrix(y_true_labels, y_pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    plt.figure(figsize=(14, 14))
    disp.plot(xticks_rotation='vertical', cmap='Blues', ax=plt.gca())
    plt.title(f"Confusion Matrix - {title}")
    plt.show()

    print(f"\nCLASSIFICATION REPORT - {title}")
    print(classification_report(y_true_labels, y_pred_labels, target_names=classes))

evaluate_generator(val_generator, title="Validation Set")

In [ ]:
# ==============================
# 8. LOAD EXTERNAL TEST DATA
# ==============================
def load_external_test_data(test_dir, img_size=(128, 128)):
    images, labels = [], []
    
    if not os.path.exists(test_dir):
        print(f"Error: Directory {test_dir} not found.")
        return np.array([]), np.array([])

    for folder in os.listdir(test_dir):
        if folder not in classes:
            continue
            
        label_idx = classes.index(folder)
        folder_path = os.path.join(test_dir, folder)
        
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            
            img = cv2.imread(img_path)
            
            if img is None:
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)
            
            # ResNet Preprocessing
            img = img.astype('float32')
            img = preprocess_input(img) 
            
            images.append(img)
            labels.append(label_idx)
            
    images = np.array(images)
    labels = to_categorical(labels, num_classes=num_classes)
    
    print(f"\nLoaded {images.shape[0]} external test images. Shape = {images.shape}")
    return images, labels

x_test_external, y_test_external = load_external_test_data(external_test_dir, img_size=input_shape[:2])



In [ ]:
# ==============================
# 9. EVALUATE ON EXTERNAL TEST SET
# ==============================
def evaluate_on_arrays(x_data, y_data, title="External Test Set"):
    if x_data.size == 0:
        print("No data to evaluate.")
        return

    loss, acc = model.evaluate(x_data, y_data, verbose=1)
    print(f"\n{title} - Accuracy: {acc:.4f}, Loss: {loss:.4f}")

    y_pred = model.predict(x_data, verbose=1)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_data, axis=1)

    cm = confusion_matrix(y_true_labels, y_pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    plt.figure(figsize=(14, 14))
    disp.plot(xticks_rotation='vertical', cmap='Blues', ax=plt.gca())
    plt.title(f"Confusion Matrix - {title}")
    plt.show()

    print(f"\nCLASSIFICATION REPORT - {title}")
    print(classification_report(y_true_labels, y_pred_labels, target_names=classes))

evaluate_on_arrays(x_test_external, y_test_external, title="External Test Set")